In [26]:
import pandas as pd

from mogestpy.quantity.Hydrological import SMAP
from spotpy.objectivefunctions import mae, rmse, nashsutcliffe, kge, pbias
from scipy.optimize import Bounds

In [27]:
pd.options.plotting.backend = "plotly"

In [28]:
df = pd.read_excel(r'D:\Dados\smap_jundiai_chuva.xlsx')
display(df)

,ETP,P,Qobs,Qplan,idx,P4d,P5d,P6d,P7d,P8d,...,P11d,P12d,P13d,P14d,P15d,P30d,P60d,1619.8433333333337,1619.8433333333346,1.0000000000000007
0,5.266667,0.000000,1.69,-0.323000,1,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
1,5.266667,0.000000,0.06,-0.319765,2,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
2,5.266667,1.100000,0.41,-0.316562,3,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
3,5.266667,0.000000,0.06,-0.313391,4,17.800476,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
4,5.266667,0.000000,1.94,-0.310251,5,0.000000,22.189635,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,4.433333,3.740000,3.50,0.449669,361,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
361,4.433333,2.696667,3.83,0.326139,362,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
362,4.433333,15.610000,7.14,3.860293,363,0.000000,0.000000,0.0,0.000000,0.0,...,49.086162,0.0,0.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
363,4.433333,6.446667,3.03,2.425178,364,17.800476,0.000000,0.0,31.150833,0.0,...,0.000000,0.0,57.851548,62.301667,0.0,0.0,0.0,NaN,NaN,NaN


In [29]:
ax = df[df.columns[5:-3]].plot.bar(barmode='group')
ax.show()

In [30]:
bacia = SMAP.SMAP.Basin(
  Str = 160.57994187826057,
  Crec = 6.272853170890949,
  Capc = 31.736891399641486,
  kkt = 46.41960215641171,
  k2t = 1.5666751507659338,
  Ai = 2.00000001,
  TUin = 0.0,
  EBin = 11.300530253519952,
  AD = 109.08)

In [31]:
df_2 = pd.DataFrame()
df_2['Qobs'] = df.Qobs

In [32]:
ponto_ref = SMAP.SMAP.Point(df.P, df.ETP)

In [33]:
modelo_ref = SMAP.SMAP(ponto_ref, bacia)


In [34]:
calib = modelo_ref.Calibrate(df.Qobs, optimization_engine='de')
print(bacia)

SMAP Basin Object:
  Str = 160.5944213848285,
  Crec = 12.452336068860363,
  Capc = 45.25152792822873,
  kkt = 125.84951389128122,
  k2t = 1.566636647646836,
  Ai = 2.00000001,
  TUin = 0.0,
  EBin = 8.732526847520429,
  AD = 109.08


In [35]:
# modelo_ref.RunModel()

In [36]:
df_2['Q_ref'] = modelo_ref.Q

In [37]:
df_2[['Qobs', 'Q_ref']].plot()

In [38]:
df.columns[5:-3]

Index(['P4d', 'P5d', 'P6d', 'P7d', 'P8d', 'P9d', 'P10d', 'P11d', 'P12d',
       'P13d', 'P14d', 'P15d', 'P30d', 'P60d'],
      dtype='object')

In [39]:
for column in df.columns[5:-3]:
    p = SMAP.SMAP.Point(df[column], df.ETP)
    modelo = SMAP.SMAP(p, bacia)
    modelo.RunModel()
    df_2[column.replace('P', 'Q')] = modelo.Q

In [40]:
df_2.plot()

In [41]:
print(f'NSE: {nashsutcliffe(df_2.Qobs, df_2.Q_ref)}')
print(f'KGE: {kge(df_2.Qobs, df_2.Q_ref)}')
print(f'PBIAS: {pbias(df_2.Qobs, df_2.Q_ref)}')
print(f'RMSE: {rmse(df_2.Qobs, df_2.Q_ref)}')

NSE: 0.6030165993760289
KGE: 0.6514334372747007
PBIAS: -30.606256282944134
RMSE: 2.1373582362280574


In [42]:

df_2.to_excel(r'D:\Dados\smap_jundiai_vazao.xlsx', index=False)

In [43]:
df_2

,Qobs,Q_ref,Q4d,Q5d,Q6d,Q7d,Q8d,Q9d,Q10d,Q11d,Q12d,Q13d,Q14d,Q15d,Q30d,Q60d
0,1.69,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.41,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.06,0.000000,0.638858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.94,0.000000,0.410444,1.017763,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,3.50,1.147684,1.473194,1.919658,2.497298,0.795930,3.630491,4.250602,4.899904,0.162670,6.268716,0.130563,0.092870,8.497488,21.389379,52.066880
361,3.83,0.749064,0.946475,1.233313,1.604427,0.511357,2.332463,2.730863,3.148017,0.104510,4.027431,0.083882,0.059666,5.459339,13.741929,33.451151
362,7.14,3.001472,0.608077,0.792361,1.030788,0.328529,1.498526,1.754484,2.022491,8.486366,2.587484,0.053892,0.038333,3.507435,8.828709,21.491196
363,3.03,2.412894,2.254095,0.509064,0.662245,4.575376,0.962751,1.127195,1.299380,5.452194,1.662368,10.587428,11.710968,2.253404,5.672137,13.807342
